### This script simply checks for two cases
#### Case-1) the two images are duplicate or not. 
#### Case-2) To find the duplicates for the given image

In [2]:
import cv2
import numpy as np
import json
import pickle
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from scipy.spatial import distance
import os,glob

In [3]:
def get_model():
    base_model = VGG16(weights='imagenet', include_top=True)
    model = Model(inputs=base_model.input,
                  outputs=base_model.get_layer('fc2').output)
    return model

def feature_extraction(img,model):
    img = cv2.resize(img,(224,224))
    img = image.img_to_array(img)
    img4d = np.expand_dims(img, axis=0)
    pred = model.predict(img4d)[0,:]
    return pred

def comparator(image1, image2, model):
    feat1 = feature_extraction(image1, model)
    feat2 = feature_extraction(image2, model)
    a = distance.euclidean(feat1, feat2)
    if a != 0:
        return False
    else:
        return True

In [5]:
a = cv2.imread("./my_try/my_try0_1.jpg")
b = cv2.imread("./my_try/my_try0_1.jpg")
c = cv2.imread("./my_try/my_try10_1.jpg")

# del model

model = get_model()

check_a_b = comparator(a,b,model)
if check_a_b == True:
    print("Duplicate")
else:
    print("Non-Duplicate")

print('****************************')

check_b_c = comparator(b,c,model)
if check_b_c == True:
    print("Duplicate")
else:
    print("Non-Duplicate")

del model    

Duplicate
****************************
Non-Duplicate


In [6]:
with open('./precompute/tunic/tunic.pickle', 'rb') as f:
    b = pickle.load(f)
    pid = list(b.keys())
    feats = list(b.values())
                

In [7]:
path_to_img = "./Tunics/TUNEJFR9SYD77PV2.jpg"

In [8]:
img = cv2.imread(path_to_img)
names = path_to_img.split("/")
fhead,_ = os.path.splitext(names[-1])
model = get_model()
feat_img = feature_extraction(img, model)
del model
final = {}
dups = []
for i, feat in enumerate(feats):
    dist = distance.euclidean(feat,feat_img)
    if dist == 0:
        dups.append(pid[i])

final[fhead] = dups

In [9]:
print(json.dumps(final, indent=5))

{
     "TUNEJFR9SYD77PV2": [
          "TUNEJFR9SYD77PV2",
          "TUNEJFR9GFQZSUGB",
          "TUNEJFR9BANCWWYK",
          "TUNEJFR9PZEHTGJX"
     ]
}


In [10]:
j = json.dumps(final)
f = open("./results/single_dump_check/"+str(fhead)+".json","w")
f.write(j)
f.close()
